In [1]:
import sys
import re
import requests
from tqdm.auto import tqdm 

from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts.base import BaseOutputParser

sys.path.insert(0, r'C:\Users\HARSH\Documents\chatweb3-backend')
from utils import *
set_api_key()

In [2]:
TEMPLATE = \
'''
Query: {query}
Note: The query is related to the blockchain ecosystem.
Generate 10 related searches most asked on the web. 
1.'''

class ChatOutputParser(BaseOutputParser):

    @property
    def _type(self) -> str:
        """Return the type key."""
        return "chat_output_parser"

    def parse(self, text: str) -> str:
        """Parse the output of an LLM call."""
        text = text.strip()
        text = re.sub(r"\d+. ", '', text)
        text = text.split('\n')
        return text

In [3]:
class SearchURLs():
    def __init__(self,) -> None:
        super().__init__()
        llm = OpenAI(
            temperature=1.0, max_tokens=-1,
        )
        output_parser = ChatOutputParser()
        relSearch_prompt = PromptTemplate(
            input_variables=["query"],
            template=TEMPLATE,
            output_parser=output_parser,
        )
        self.chain = LLMChain(llm=llm, prompt=relSearch_prompt, verbose=False)

        subscription_key="c6279a2b2a41436caa61fe104e464404"
        self.search_url = "https://api.bing.microsoft.com/v7.0/search"
        self.headers = {"Ocp-Apim-Subscription-Key" : subscription_key}

    def via_llm(self, query): 
        relSearches = self.chain.apply_and_parse([{"query": query}])[0]
        return relSearches

    def via_bing(self, query):
        query = query+" Note: The query is related to the blockchain ecosystem."
        params  = {"q": query, "textDecorations": True, "textFormat": "HTML"}
        response = requests.get(self.search_url, headers=self.headers, params=params)
        response.raise_for_status()
        response = response.json()
        urls = [x['url'] for x in response['webPages']['value']]
        try: relSearches = [r['text'] for r in response['relatedSearches']['value']]
        except: relSearches = []
        return urls, relSearches

In [4]:
with open("init_questions4.txt") as file:
    queries = [line.rstrip() for line in file]

In [5]:
def find_urls(query, searcher, depth):
    urls, relS_bing = searcher.via_bing(query)
    urls = [u for u in urls if u not in all_urls]
    all_urls.update(set(urls))
    relS_llm = searcher.via_llm(query)
    if depth==0: return {query: urls}
    relS = relS_bing + relS_llm
    q_urls = []
    for q in relS:
        if q in all_queries: continue
        x = find_urls(q, searcher, depth=depth-1)
        all_queries.add(q)
        q_urls.append(x)
    return q_urls

In [ ]:
searcher = SearchURLs()
all_queries, all_urls = set(), set()
for q in tqdm(queries):
    urls = find_urls(q, searcher, depth=2)

In [ ]:
with open("urls2scrape2.txt", 'w') as output:
    for row in urls:
        output.write(str(row) + '\n')